In [ ]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
mpl.rcParams['figure.figsize'] = [15, 6]

In [ ]:
notebook_name = Path("trial_data.ipynb")

In [ ]:
base_folder = Path(".").resolve().parent
result_folder = Path(".")/notebook_name.stem
result_folder.mkdir(exist_ok=True, parents=True)
event_step = Step(Path(".").resolve() / "events.ipynb")
fiber_file = singleglob(base_folder, "fiber/Fluorescence.csv")
result_path =  result_folder / "trial_data.nc"
base_folder, fiber_file, event_step

In [ ]:
tables = TableWriter(result_folder/"tables.xlsx")
figures = PdfWriter(result_folder/"figures.pdf")
dicts = DictWriter(result_folder/"dicts.yaml")
notebook_save_path = result_folder/"notebook.html"
warn =  TextWriter(result_folder/"warnings.txt")


In [ ]:
events = xr.load_dataset(event_step.exec_if_necessary()/"trial_events.nc")
events

In [ ]:
fiber_data = pd.read_csv(fiber_file, skiprows=1).dropna(axis=1, how="all")
fiber_data.insert(0, "t", fiber_data["TimeStamp"]/1000)
fiber_data = fiber_data.drop(columns=["TimeStamp", "Events"])
fiber_data


In [ ]:
stacked = fiber_data.set_index("t").rename_axis("channel", axis=1).stack().to_frame("value").reset_index()
stacked["channel_hemi"] = stacked["channel"].str.extract('CH(\d*)-').astype(int)
stacked["channel_hemi"] = stacked["channel_hemi"].map({1: "Left", 2:"Right"})
stacked["channel_type"] = stacked["channel"].str.extract('CH\d*-(\d*)').astype(int)
stacked["channel_type"] = stacked["channel_type"].map({410: "baseline", 470:"dlight"})
dataset = stacked.set_index(["t", "channel_hemi", "channel_type"])["value"].to_xarray().to_dataset(name="raw_value")
dataset

In [ ]:
dataset["df/f"] = (dataset["raw_value"].sel(channel_type="dlight") - dataset["raw_value"].sel(channel_type="baseline"))/dataset["raw_value"].sel(channel_type="baseline")
dataset

In [ ]:
dataset["t_trial"] = xr.DataArray(np.arange(-3, 10, step=0.020), dims="t_trial")
dataset["aligned_df/f"] = dataset["df/f"].interp(t=events["event_t"]+dataset["t_trial"])
dataset

In [ ]:
zscore_window = dataset["aligned_df/f"].sel(event_name="cue", t_trial=slice(-1, 0))
dataset["zscored"] =(dataset["aligned_df/f"] - zscore_window.mean("t_trial"))/zscore_window.std("t_trial")
dataset


In [ ]:
dataset = xr.merge([dataset, events])
dataset

In [ ]:
dataset["rt"] = dataset["event_t"].sel(event_name="mouvement_start") - dataset["event_t"].sel(event_name="cue")
dataset["mt"] = dataset["event_t"].sel(event_name="leverpress") - dataset["event_t"].sel(event_name="mouvement_start")
dataset["sucess"] = dataset["event_t"].sel(event_name = "error").isnull() & dataset["rt"].notnull() & dataset["mt"].notnull() & (dataset["rt"] < 2) & (dataset["mt"] < 2)

In [ ]:
res = dataset.drop_dims("t").drop_vars("aligned_df/f")
res.to_netcdf(result_path)
xr.load_dataset(result_path)

# Finishing

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
import os
os.system(f'jupyter nbconvert --to html {notebook_name} --output {notebook_save_path} --no-prompt --Application.log_level=40')


In [ ]:
del tables
del dicts
del figures
del warn